In [4]:
import librosa
import pickle

with open("their_model/their_tuned_model.pkl", 'rb') as file:
    pickle_model = pickle.load(file)

from joblib import load
scaler = load('their_model/std_scaler.bin')

def score_prob_of_being(y, sr=44100, being=None):
    rms = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
    for e in mfcc:
        to_append += f' {np.mean(e)}'

    df = np.array([[float(x) for x in to_append.split(" ")]])
    
    X = scaler.transform(df)
    
    instr_list = ["flu", "gac", "org", "pia", "tru", "voi"]
    
    if being:
        return pickle_model.predict_proba(X)[0][instr_list.index(being)]
    else:
        return pickle_model.predict_proba(X)[0]

In [5]:
songname = "../IRMAS-TrainingData/flu/008__[flu][nod][cla]0393__1.wav"
y, sr = librosa.load(songname, sr=44100)
print(score_prob_of_being(y, sr, "flu"))

C:\Users\scgry\AppData\Local\Temp\ipykernel_11808\4294197670.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(songname, sr=44100)
F:\sonic-subterfuge\venv\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '../IRMAS-TrainingData/flu/008__[flu][nod][cla]0393__1.wav'

In [166]:
import numpy as np
from deap import base, creator, tools, algorithms

# Define the problem: Maximizing the fitness of the spectrogram
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

# Define the functions for initialization, mutation, crossover, and evaluation
def init_individual():
    songname = "../IRMAS-TrainingData/flu/008__[flu][nod][cla]0393__1.wav"
    y, sr = librosa.load(songname, sr=44100)
    return np.real(librosa.stft(y))

def mutate(individual, indpb):
    # Mutate an individual by adding a small random value to each element
    individual += np.random.normal(0, indpb, individual.shape)
    return individual,

def crossover(parent1, parent2):
    # Perform crossover by taking the average of corresponding elements
    return (parent1 + parent2) / 2,

def evaluate(individual):
    # Your evaluation function - this would be where you compute the fitness of the spectrogram
    # Example fitness function: maximize the sum of the spectrogram values
    return score_prob_of_being(np.real(librosa.istft(individual)), 44100, "tru"),

# Set up the DEAP framework
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Create an initial population
population_size = 500
population = toolbox.population(n=population_size)

# Run the evolution
generations = 100
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)
population, logbook = algorithms.eaMuPlusLambda(population, toolbox, mu=population_size, lambda_=population_size*2,
                                                cxpb=0.7, mutpb=0.2, ngen=generations, stats=stats, halloffame=None, verbose=True)

C:\Users\scgry\IdeaProjects\musical-instrument-classification\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\scgry\IdeaProjects\musical-instrument-classification\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg       	max       
0  	500   	0.00285597	0.00285597
1  	895   	0.0997367 	0.267972  
2  	904   	0.238463  	0.274427  
3  	900   	0.272394  	0.275536  
4  	885   	0.274172  	0.27968   
5  	907   	0.274717  	0.27968   
6  	895   	0.275054  	0.284336  
7  	899   	0.275509  	0.291471  
8  	890   	0.276068  	0.293662  
9  	895   	0.27762   	0.295615  
10 	908   	0.279949  	0.296733  
11 	894   	0.283882  	0.300345  
12 	902   	0.289036  	0.300669  
13 	906   	0.293043  	0.300771  
14 	902   	0.29592   	0.304941  
15 	897   	0.298183  	0.30495   
16 	909   	0.299741  	0.30495   
17 	906   	0.301133  	0.306852  
18 	889   	0.302507  	0.306852  
19 	897   	0.304018  	0.307549  
20 	917   	0.305103  	0.308239  
21 	901   	0.305933  	0.308945  
22 	903   	0.306674  	0.310606  
23 	901   	0.307388  	0.311073  
24 	906   	0.307868  	0.311829  
25 	900   	0.30845   	0.311852  
26 	892   	0.309395  	0.311977  
27 	904   	0.310261  	0.312161  
28 	895   	0.310889  	0.312609  
29 	894   

In [160]:
# Access the best individual after evolution
best_individual = tools.selBest(population, k=1)[0]
best_fitness = best_individual.fitness.values[0]

print("Best Individual:", best_individual)
print(score_prob_of_being(np.real(librosa.istft(best_individual)), 44100))
print("Best Fitness:", best_fitness)

Best Individual: [[-0.43808126 -0.3202868   0.19258332 ...  0.0514356  -0.20646
  -1.0711545 ]
 [ 0.45840842 -0.05321011  0.00810173 ... -0.07512452 -0.12301416
   1.0700948 ]
 [-0.21473972  0.74139744 -0.30315354 ...  0.27968916  0.19839843
  -0.4522809 ]
 ...
 [ 0.08252966  0.00391369 -0.11209022 ...  0.04768083 -0.19643939
  -0.02393327]
 [ 0.19777937  0.00437645 -0.286534   ...  0.02452982  0.08016134
  -0.11615957]
 [ 0.08018901 -0.07383145 -0.20534149 ... -0.09224464 -0.10221875
  -0.04054805]]
[0.2970134  0.05134434 0.16856491 0.07360226 0.27491982 0.13455527]
Best Fitness: 0.27491981653575875


In [161]:
from IPython.lib.display import Audio

out = librosa.istft(best_individual)
print(len(out))
Audio(data=librosa.istft(init_individual()), rate=44100)
Audio(data=out, rate=44100)

132096


KeyboardInterrupt: 